# Week4 授業課題 信用情報の学習

## 1.このテキストについて

### テキストの目的
- 実データに近いものに対して機械学習を行う

### どのように学ぶか
- Kaggleのデータセットを通じて、機械学習の基本的な流れやライブラリの使い方を学びます。

## 2.信用情報
Kaggleの Home Credit Default Risk コンペティションの学習および推定を行います。

Home Credit Default Risk | Kaggle

### 【問題1】コンペティション内容の確認
コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

- 何を学習し、何を予測するのか
- どのようなファイルを作りKaggleに提出するか
- 提出されたものはどういった指標値で評価されるのか

### 【解答】
- 顧客の収入などの個人データから貸し倒れのリスクを予測する
- 顧客IDごとの貸し倒れの確率をCSVファイルにして提出する
- ROC曲線で評価する

## 3.ベースラインモデルの作成
まずは単純な方法による ベースラインモデル　を作成します。精度の基準となるモデルです。

精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。

### 【問題2】学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。

検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")

In [4]:
display(train_df.head())
display(test_df.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#欠損値がある列を削除
train_drop_df = train_df.dropna(axis=1)
test_drop_df = test_df.dropna(axis=1)

In [6]:
#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

In [7]:
X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

In [8]:
# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

In [10]:
forest = RandomForestClassifier(n_estimators=10)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
y_val_pred = forest.predict(X_val)

In [14]:
def evaluate(y_test, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.919781992247457
precision : 0.31
recall : 0.010178952552946971
f1 : 0.019710697822285805
confusion matrix 
 [[70649   138]
 [ 6029    62]]


### 【問題3】テストデータに対する推定
テストデータ（application_test.csv）に対して推定を行い、Kaggleに提出を行ってください。

正しく提出が行えていれば、精度は低くても構いません。

In [15]:
y_pred = forest.predict(X_test)

## 4.特徴量エンジニアリング
ベースラインモデルを元に、入力する特徴量に様々な工夫を行い精度を向上させていきます。

### 【問題4】特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。

- どの特徴量を使うか
- どう前処理をするか

何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。

精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。

- 欠損地を0で埋める

In [16]:
train_drop_df = train_df.fillna(0)
test_drop_df = test_df.fillna(0)

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.9197299617575899
precision : 0.3
recall : 0.009850599244787392
f1 : 0.019074868860276588
confusion matrix 
 [[70647   140]
 [ 6031    60]]


- 欠損値を平均値で埋める

In [ ]:
train_drop_df = train_df.fillna(train_df.mean())
test_drop_df = test_df.fillna(test_df.mean())

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

- 欠損値を中央値で埋める

In [ ]:
train_drop_df = train_df.fillna(train_df.median())
test_drop_df = test_df.fillna(test_df.median())

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

- 欠損値を最頻値で埋める

In [ ]:
train_drop_df = train_df.fillna(train_df.mode())
test_drop_df = test_df.fillna(test_df.mode())

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

- 重要そうな特徴量を選んで残りは削除する

In [27]:
#objectタイプの列を削除する
select_columns = ["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY"]
train_drop_df = train_df[select_columns]
test_drop_df = test_df[select_columns]

train_drop_df = train_drop_df.dropna(axis=1)
test_drop_df = test_drop_df.dropna(axis=1)

X = train_drop_df
y = train_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.9104165040713859
precision : 0.10515873015873016
recall : 0.017402725332457725
f1 : 0.029863361036765745
confusion matrix 
 [[69885   902]
 [ 5985   106]]
